Changes made by @fhalamos from https://github.com/jmather625/predicting-poverty-replication

Make sure you download the 2016 Household LSMS survey data for Malawi, Ethiopia, and Nigeria from https://microdata.worldbank.org/index.php/catalog/lsms and put it in `../data/countries/`. Malawi's data should be named `malawi_2016/LSMS`, Ethiopia's should be named `ethiopia_2015/LSMS`, and Nigeria's should be named `nigeria_2015/LSMS`. Nightlights data should be downloaded from https://ngdc.noaa.gov/eog/viirs/download_dnb_composites.html using the annual composite from 2015 in tile 2 and tile 5.

In [1]:
! pip install geoio
! pip install utils

     |████████████████████████████████| 61kB 5.6MB/s 
     |████████████████████████████████| 23.7MB 56.8MB/s 
  Created wheel for tzwhere: filename=tzwhere-3.0.3-cp37-none-any.whl size=23742584 sha256=b58d1a6cc0a6b2d5014a7e3dc2996c0be681cadd5164c80725a58ce83838b173
  Stored in directory: /root/.cache/pip/wheels/89/18/2f/01c958c82b1223d6fe763c6b2fadf45b4f6ce6e8a9fce7bbd5
Successfully built tzwhere


In [2]:
import pandas as pd
import numpy as np
import os
import geoio

In [57]:
from google.colab import files, drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
!pwd

/content


In [6]:
BASE_DIR = '/content/drive/MyDrive/Data Science GRDS [Felipe]/Poverty Measurement'

In [7]:

NIGHTLIGHTS_DIRS = [os.path.join(BASE_DIR, 'data/nightlights/viirs_2015_00N060W.tif'),
                    os.path.join(BASE_DIR, 'data/nightlights/viirs_2015_75N060W.tif')]

COUNTRIES_DIR = os.path.join(BASE_DIR, 'data', 'countries')

In [8]:
import sys
sys.path.append(BASE_DIR)
# from utils import create_space

In [28]:
def process_country_data(country_abbreviation, lsms_dir, consumption_file, hhsize_col, geovariables_file, lat_col, lon_col, ppp, hh_id_col, consumption_ph_col=None, consumption_pc_col=None):
    '''
    consumption_ph_col # per household
    consumption_pc_col # per capita
    '''
    #Check files exist
    for file in [consumption_file, geovariables_file]:       
        file_path = os.path.join(lsms_dir, file)
        assert os.path.isfile(file_path), print(f'Could not find {file_path}')
    
    #Load consumption df    
    df = pd.read_csv(os.path.join(lsms_dir, consumption_file))
    
    #Create consumption per household column
    if consumption_ph_col is not None:
      df['cons_ph'] = df[consumption_ph_col]
    #If consumption was reported per capita, multiply by hh size to get per household
    elif consumption_pc_col is not None:
      df['cons_ph'] = df[consumption_pc_col] * df[hhsize_col]

    #Create people per household column
    df['pph'] = df[hhsize_col]

    # Create consumption per household adjusted pp
    df['cons_ph'] = df['cons_ph'] / ppp / 365

    #Filter df to only selected columns
    df = df[[hh_id_col, 'cons_ph', 'pph']]

    #Load geovariables df
    df_geo = pd.read_csv(os.path.join(lsms_dir, geovariables_file))
    df_cords = df_geo[[hh_id_col, lat_col, lon_col]]

    #Rename lat lon columns after cluster
    df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)

    #Merge geodata with consumption data on hh identifier
    df_combined = pd.merge(df, df_cords, on=hh_id_col)

    #Drop repeted identifier
    df_combined.drop([hh_id_col], axis=1, inplace=True)

    #Drop na values
    df_combined.dropna(inplace=True)

    #Aggregate consumption at the cluster level
    df_clusters = df_combined.groupby(['cluster_lat', 'cluster_lon']).sum().reset_index()

    #Compute consumption per capita
    df_clusters['cons_pc'] = df_clusters['cons_ph'] / df_clusters['pph'] # divides total cluster income by people
    df_clusters['country'] = country_abbreviation
    return df_clusters[['country', 'cluster_lat', 'cluster_lon', 'cons_pc']]


In [30]:
#Countries parameters
countries_params = {}

#Malawi parameters
country_params = {}
country_params['lsms_dir'] = os.path.join(COUNTRIES_DIR, 'malawi_2016', 'LSMS')
country_params['consumption_file'] = os.path.join('consumption_aggregate','ihs4 consumption aggregate.csv')
country_params['consumption_pc_col'] = None
country_params['consumption_ph_col'] = 'rexpagg' # per household
country_params['hhsize_col'] = 'hhsize' # people in household
country_params['geovariables_file'] = os.path.join('household_geovariables','householdgeovariablesihs4.csv')
country_params['lat_col'] = 'lat_modified'
country_params['lon_col'] = 'lon_modified'
country_params['ppp'] = 215.182 # purchasing power parity for malawi in 2016 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=MW)
country_params['hh_id_col'] = 'case_id'
country_params['country_abbreviation'] = 'mw'
countries_params['mw'] = country_params

#Ethiopia parameters
country_params = {}
country_params['lsms_dir'] = os.path.join(COUNTRIES_DIR, 'ethiopia_2015', 'LSMS')
country_params['consumption_file'] = 'Consumption Aggregate/cons_agg_w3.csv'
country_params['consumption_pc_col'] = 'total_cons_ann'
country_params['consumption_ph_col'] = None
country_params['hhsize_col'] = 'hh_size' # people in household
country_params['geovariables_file'] = 'Geovariables/ETH_HouseholdGeovars_y3.csv'
country_params['lat_col'] = 'lat_dd_mod'
country_params['lon_col'] = 'lon_dd_mod'
country_params['ppp'] = 7.882 # purchasing power parity for malawi in 2016 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=MW)
country_params['hh_id_col'] = 'household_id2'
country_params['country_abbreviation'] = 'eth'
countries_params['eth'] = country_params

#Nigeria parameters
country_params = {}
country_params['lsms_dir'] = os.path.join(COUNTRIES_DIR, 'nigeria_2015', 'LSMS')
country_params['consumption_file'] = 'cons_agg_wave3_visit1.csv'
country_params['consumption_pc_col'] = 'totcons' # per capita
country_params['consumption_ph_col'] = None
country_params['hhsize_col'] = 'hhsize' # people in household
country_params['geovariables_file'] = 'nga_householdgeovars_y3.csv'
country_params['lat_col'] = 'LAT_DD_MOD'
country_params['lon_col'] = 'LON_DD_MOD'
country_params['ppp'] = 95.255 # purchasing power parity for nigeria in 2015 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=NG)
country_params['hh_id_col'] = 'hhid'
country_params['country_abbreviation'] = 'ng'
countries_params['ng'] = country_params

In [43]:
#Load countries data

countries_dfs = {}
df_mw = None
df_eth = None
df_ng = None

for country_abbrv in ['mw', 'eth', 'ng']:
    countries_dfs[country_abbrv] = process_country_data(
        country_abbreviation = country_abbrv,
        lsms_dir = countries_params[country_abbrv]['lsms_dir'],
        consumption_file = countries_params[country_abbrv]['consumption_file'],
        hhsize_col = countries_params[country_abbrv]['hhsize_col'],
        geovariables_file = countries_params[country_abbrv]['geovariables_file'],
        lat_col = countries_params[country_abbrv]['lat_col'],
        lon_col = countries_params[country_abbrv]['lon_col'],
        ppp = countries_params[country_abbrv]['ppp'],
        hh_id_col = countries_params[country_abbrv]['hh_id_col'],
        consumption_ph_col=countries_params[country_abbrv]['consumption_ph_col'],
        consumption_pc_col=countries_params[country_abbrv]['consumption_pc_col'])

(780, 4)
(523, 4)
(664, 4)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [46]:
tifs = [geoio.GeoImage(ndir) for ndir in NIGHTLIGHTS_DIRS]

2

In [55]:
# loading both of these into memory requires A LOT of free memory (at least 4 gigs)
# using a swapfile of size 2 GB still did not fix my issues
# instead, I knew ahead of time the 0th tif is for Malawi, and the 1st tif is for Ethiopia and Nigeria
# I'll use this to only load one tif at a time
# thankfully, the countries did not span across two tifs
tif_array = np.squeeze(tifs[0].get_data())

In [ ]:
def add_nightlights(df, tif, tif_array):
    ''' 
    This takes a dataframe with columns cluster_lat, cluster_lon and finds the average 
    nightlights in 2015 using a 10kmx10km box around the point
    
    I try all the nighlights tifs until a match is found, or none are left upon which an error is raised
    '''
    cluster_nightlights = []
    for i,r in df.iterrows():
        min_lat, min_lon, max_lat, max_lon = create_space(r.cluster_lat, r.cluster_lon)
        
        xminPixel, ymaxPixel = tif.proj_to_raster(min_lon, min_lat)
        xmaxPixel, yminPixel = tif.proj_to_raster(max_lon, max_lat)
        assert xminPixel < xmaxPixel, print(r.cluster_lat, r.cluster_lon)
        assert yminPixel < ymaxPixel, print(r.cluster_lat, r.cluster_lon)
        if xminPixel < 0 or xmaxPixel >= tif_array.shape[1]:
            print(f"no match for {r.cluster_lat}, {r.cluster_lon}")
            raise ValueError()
        elif yminPixel < 0 or ymaxPixel >= tif_array.shape[0]:
            print(f"no match for {r.cluster_lat}, {r.cluster_lon}")
            raise ValueError()
        xminPixel, yminPixel, xmaxPixel, ymaxPixel = int(xminPixel), int(yminPixel), int(xmaxPixel), int(ymaxPixel)
        cluster_nightlights.append(tif_array[yminPixel:ymaxPixel,xminPixel:xmaxPixel].mean())
        
    df['nightlights'] = cluster_nightlights

In [ ]:
add_nightlights(df_mw, tifs[0], tif_array)

In [ ]:
del tif_array
import gc
gc.collect()

5403

In [ ]:
import psutil
psutil.virtual_memory()

svmem(total=3883982848, available=3294400512, percent=15.2, used=440209408, free=3389562880, active=332414976, inactive=85729280, buffers=3567616, cached=50642944, shared=1142784, slab=24154112)

In [ ]:
tif_array = np.squeeze(tifs[1].get_data())

In [ ]:
add_nightlights(df_eth, tifs[1], tif_array)

In [ ]:
add_nightlights(df_ng, tifs[1], tif_array)

In [ ]:
df_mw.head()

,country,cluster_lat,cluster_lon,cons_pc,nightlights
0,mw,-17.095150,35.217213,1.423239,0.025206
1,mw,-17.092351,35.114643,1.266204,0.000000
2,mw,-17.016698,35.079629,1.566870,0.000000
3,mw,-16.977243,35.205706,1.669245,0.008266
4,mw,-16.956385,35.168967,1.089891,0.002295


In [ ]:
df_eth.head()

,country,cluster_lat,cluster_lon,cons_pc,nightlights
0,eth,3.455701,39.515994,14.854634,0.0
1,eth,3.549937,39.184234,14.312022,0.0
2,eth,3.864243,39.101366,12.470145,0.0
3,eth,3.982931,38.491368,8.336539,0.0
4,eth,4.048194,41.930928,4.762526,0.0


In [ ]:
df_ng.head()

,country,cluster_lat,cluster_lon,cons_pc,nightlights
0,ng,4.315786,6.268753,4.317717,0.123354
1,ng,4.328719,6.308172,4.880711,0.013713
2,ng,4.398427,7.183962,8.767258,38.470989
3,ng,4.425192,7.166935,10.774504,40.519035
4,ng,4.619377,7.684946,5.191333,0.000000


In [ ]:
df_mw['nightlights'].mean()

0.6038162894313969

In [ ]:
df_eth['nightlights'].mean()

0.6727544504689029

In [ ]:
df_ng['nightlights'].mean()

1.6584013095848769

In [ ]:
df_mw.corr()

,cluster_lat,cluster_lon,cons_pc,nightlights
cluster_lat,1.000000,-0.702793,-0.026563,-0.083273
cluster_lon,-0.702793,1.000000,-0.002947,-0.033367
cons_pc,-0.026563,-0.002947,1.000000,0.384939
nightlights,-0.083273,-0.033367,0.384939,1.000000


In [ ]:
df_eth.corr()

,cluster_lat,cluster_lon,cons_pc,nightlights
cluster_lat,1.000000,0.157130,-0.052673,-0.006627
cluster_lon,0.157130,1.000000,0.211846,0.076039
cons_pc,-0.052673,0.211846,1.000000,0.343879
nightlights,-0.006627,0.076039,0.343879,1.000000


In [ ]:
df_ng.corr()

,cluster_lat,cluster_lon,cons_pc,nightlights
cluster_lat,1.000000,0.330910,-0.376356,-0.188281
cluster_lon,0.330910,1.000000,-0.246189,-0.196671
cons_pc,-0.376356,-0.246189,1.000000,0.264398
nightlights,-0.188281,-0.196671,0.264398,1.000000


In [ ]:
for country in ['malawi_2016', 'ethiopia_2015', 'nigeria_2015']:
    os.makedirs(os.path.join(COUNTRIES_DIR, country, 'processed'), exist_ok=True)

In [ ]:
df_mw.to_csv(os.path.join(COUNTRIES_DIR, 'malawi_2016', 'processed/clusters.csv'), index=False)

In [ ]:
df_eth.to_csv(os.path.join(COUNTRIES_DIR, 'ethiopia_2015', 'processed/clusters.csv'), index=False)

In [ ]:
df_ng.to_csv(os.path.join(COUNTRIES_DIR, 'nigeria_2015', 'processed/clusters.csv'), index=False)